# OpenDrift

### Documentation

[https://github.com/opendrift/opendrift/wiki](https://github.com/opendrift/opendrift/wiki)

***

### Installation

`git clone https://github.com/OpenDrift/opendrift.git`

`python setup.py develop --user`

***

In [2]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
import os
from glob import glob
from datetime import datetime, timedelta
from dateutil.parser import parse
from itertools import repeat
from salishsea_tools import viz_tools
from rotate_fields_python import rotate_fields
from tqdm import tqdm_notebook as tqdm
from IPython.display import HTML

from opendrift.readers import reader_netCDF_CF_generic
from opendrift.models.oceandrift import OceanDrift

%matplotlib inline
plt.rcParams['font.size'] = 12
plt.rcParams['animation.html'] = 'html5'

***
### Prepare forcing files
Forcing must be gridded on lon/lat coordinates. The easiest way is to reshape everything onto a single spatial dimension and use `opendrift.readers.reader_netCDF_CF_unstructured`.

In [3]:
# Define paths
paths = {
    'NEMO': '/results/SalishSea/hindcast.201812',
    'NEMO_cutoff': '/results2/SalishSea/hindcast.201812_annex',
    'HRDPS': '/results/forcing/atmospheric/GEM2.5/operational',
    'out': '/data/bmoorema/results/opendrift',
    'date_cutoff': '2016 Nov 21',
}
mask = xr.open_dataset('/data/bmoorema/MEOPAR/grid/mesh_mask201702.nc')
coords = xr.open_dataset('/data/bmoorema/MEOPAR/grid/coordinates_seagrid_SalishSea201702.nc', decode_times=False)
daterange = [parse(d) for d in ['2017 Nov 10', '2017 Nov 12']]

Prefix constructor for SalishSeaCast

In [4]:
def make_prefix(date, paths, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results/SalishSea/hindcast.201812/ddmmmyy/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    path = paths['NEMO']
    if 'date_cutoff' in paths and date >= parse(paths['date_cutoff']):
        path = paths['NEMO_cutoff']
    fn = '_'.join([f'SalishSea_1{res}', *repeat(date.strftime('%Y%m%d'), 2)])
    prefix = os.path.join(path, date.strftime('%d%b%y').lower(), fn)
    
    return prefix

Calculate approximate lon/lat coordinate arrays to give similar spacing to NEMO grid_X, grid_Y

In [5]:
def calculate_coordinate_arrays(coords):
    """Calculate approximate lon and lat indices to give similar spacing as NEMO grid_X, grid_Y
    """

    lonlat = []
    for var1, var2, axis, fac in zip(['glamt', 'gphit'], ['e1t', 'e2t'], ['x', 'y'], [np.cos(49 * np.pi / 180), 1]):
        deg2m = fac * 111e3
        n = coords.dims[axis] - 1
        coord, spacing = coords[var1].values, mask[var2].values.mean()
        coord_0 = (coord.max() + coord.min()) / 2 - (n / 2) * spacing / deg2m
        lonlat.append(coord_0 + np.arange(n) * spacing / deg2m)

    return lonlat

Process SalishSeaCast

In [6]:
def process_NEMO(date, paths):
    """
    """
    
    # Process input
    prefix = make_prefix(date, paths)
    NEMO_out = os.path.join(paths['out'], 'forcing', os.path.split(prefix)[-1] + '_opendrift.nc')
    if os.path.exists(NEMO_out):
        print(f'{NEMO_out} already exists! Skipping ...')
        return
    else:
        print(f'Processing {NEMO_out}')
        raw = []
        for k, key in zip(['U', 'V'], ['vozocrtx', 'vomecrty']):
            with xr.open_dataset(prefix + f'_grid_{k}.nc') as data:
                raw.append(data[key][:, 0, ...].values)
                if k is 'U': time = data.time_counter.values.astype('datetime64[s]').astype(datetime)
        u, v = viz_tools.unstagger(*raw)
        lon, lat = calculate_coordinate_arrays(coords)
        
        # Write to netCDF out
        with nc.Dataset(NEMO_out, 'w') as ds:
            
            # Create time dimension
            time_units = 'seconds since 1900-01-01 00:00:00'
            ds.createDimension('time', None)
            tempvar = ds.createVariable('time', float, 'time', zlib=True)
            tempvar[:] = nc.date2num(time, units=time_units, calendar='gregorian')
            tempvar.standard_name = 'time'
            tempvar.calendar = 'gregorian'
            tempvar.units = time_units
            
            # Create lon/lat
            dims = ('latitude', 'longitude')
            for var, name, n in zip([lat, lon], dims, u[0, ...].shape):
                ds.createDimension(name, n)
                tempvar = ds.createVariable(name, float, name, zlib=True, fill_value=1e20)
                tempvar[:] = var
                tempvar.standard_name = name
            
            # Create velocities
            for var, name in zip([u, v], ['x_sea_water_velocity', 'y_sea_water_velocity']):
                tempvar = ds.createVariable(name, float, ('time',) + dims, zlib=True, fill_value=1e20)
                tempvar[:] = var
                tempvar.standard_name = name

In [7]:
def process_mask(paths, mask):
    """
    """
    
    # Process input
    mask_out = os.path.join(paths['out'], 'forcing', 'meshmask_opendrift.nc')
    if os.path.exists(mask_out):
        print(f'{mask_out} already exists! Skipping ...')
        return
    else:
        print(f'Processing {mask_out}')
        tmask = mask.tmask[0, 0, 1:, 1:].values
        lon, lat = calculate_coordinate_arrays(coords)
        
        # Write to netCDF out
        with nc.Dataset(mask_out, 'w') as ds:
            
            # Create time dimension
            time_units = 'seconds since 1900-01-01 00:00:00'
            ds.createDimension('time', None)
            tempvar = ds.createVariable('time', float, 'time', zlib=True)
            tempvar[:] = nc.date2num(datetime(2019, 1, 1, 0, 0, 0), units=time_units, calendar='gregorian')
            tempvar.standard_name = 'time'
            tempvar.calendar = 'gregorian'
            tempvar.units = time_units
            
            # Create lon/lat
            dims = ('latitude', 'longitude')
            for var, name, n in zip([lat, lon], dims, tmask.shape):
                ds.createDimension(name, n)
                tempvar = ds.createVariable(name, float, name, zlib=True, fill_value=1e20)
                tempvar[:] = var
                tempvar.standard_name = name
        
            # Create landmask
            tempvar = ds.createVariable('land_binary_mask', float, dims, zlib=True, fill_value=1e20)
            tempvar[:] = tmask
            tempvar.standard_name = 'land_binary_mask'

Process HRDPS

In [8]:
def process_HRDPS(date, paths):
    """
    """

    datestr = date.strftime('ops_y%Ym%md%d.nc')
    HRDPS_out = os.path.join(paths['out'], 'forcing', datestr)
    if os.path.exists(HRDPS_out):
        print(f'{HRDPS_out} already exists! Skipping ...')
        return
    else:
        print(f'Processing {HRDPS_out}')
        with xr.open_dataset(os.path.join(paths['HRDPS'], datestr)) as data:
            time = data.time_counter
            time.attrs['standard_name'] = 'time'
            xr.Dataset(
                {
                    'longitude': ('flat', data.nav_lon.values.reshape(-1)),
                    'latitude': ('flat', data.nav_lat.values.reshape(-1)),
                    'u_wind': (['time_counter', 'flat'], data.u_wind.values.reshape(time.size, -1), {'standard_name': 'x_wind'}),
                    'v_wind': (['time_counter', 'flat'], data.v_wind.values.reshape(time.size, -1), {'standard_name': 'y_wind'}),
                },
                coords={'time_counter': time},
            ).to_netcdf(HRDPS_out)

Loop processing

In [9]:
# Date range
process_mask(paths, mask)
for day in tqdm(range(np.diff(daterange)[0].days + 1)):
    date = daterange[0] + timedelta(days=day)
    process_NEMO(date, paths)
    #process_HRDPS(date, paths)

Processing /data/bmoorema/results/opendrift/forcing/meshmask_opendrift.nc


/data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc already exists! Skipping ...
/data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171111_20171111_opendrift.nc already exists! Skipping ...
/data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171112_20171112_opendrift.nc already exists! Skipping ...



In [18]:
ds = xr.open_dataset('/data/bmoorema/results/opendrift/forcing/meshmask_opendrift.nc')
ds

<xarray.Dataset>
Dimensions:           (latitude: 897, longitude: 397, time: 1)
Coordinates:
  * time              (time) datetime64[ns] 2019-01-01
  * latitude          (latitude) float64 47.8 47.81 47.81 ... 50.15 50.16 50.16
  * longitude         (longitude) float64 -124.6 -124.5 -124.5 ... -123.2 -123.2
Data variables:
    land_binary_mask  (latitude, longitude) float64 ...

In [19]:
ds = xr.open_dataset('/data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc')
ds

<xarray.Dataset>
Dimensions:               (latitude: 897, longitude: 397, time: 24)
Coordinates:
  * time                  (time) datetime64[ns] 2017-11-10T00:30:00 ... 2017-11-10T23:30:00
  * latitude              (latitude) float64 47.8 47.81 47.81 ... 50.16 50.16
  * longitude             (longitude) float64 -124.6 -124.5 ... -123.2 -123.2
Data variables:
    x_sea_water_velocity  (time, latitude, longitude) float64 ...
    y_sea_water_velocity  (time, latitude, longitude) float64 ...

***

In [20]:
o = OceanDrift(loglevel=0)

11:36:29 INFO: OpenDriftSimulation initialised (version 1.0.6)


In [21]:
datestr = daterange[0].strftime('%Y%m')
filenames = sorted(glob(os.path.join(paths['out'], 'forcing', f'SalishSea_1h_{datestr}*')))
#o.add_readers_from_list(filenames, lazy=False)
reader_landmask = reader_netCDF_CF_generic.Reader('/data/bmoorema/results/opendrift/forcing/meshmask_opendrift.nc')
reader_velocity = reader_netCDF_CF_generic.Reader(filenames[0])
o.add_reader([reader_landmask, reader_velocity])

11:36:31 INFO: Opening dataset: /data/bmoorema/results/opendrift/forcing/meshmask_opendrift.nc
11:36:31 INFO: Opening file with Dataset
11:36:31 DEBUG: Finding coordinate variables.
11:36:31 DEBUG: Parsing variable: time
11:36:31 DEBUG: Parsing variable: latitude
11:36:31 DEBUG: Parsing variable: longitude
11:36:31 DEBUG: Parsing variable: land_binary_mask
11:36:31 DEBUG: Lon and lat are 1D arrays, assuming latong projection
11:36:31 DEBUG: Setting buffer size 49 for reader /data/bmoorema/results/opendrift/forcing/meshmask_opendrift.nc, assuming a maximum average speed of 5 m/s.
11:36:31 INFO: Opening dataset: /data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc
11:36:31 INFO: Opening file with Dataset
11:36:31 DEBUG: Finding coordinate variables.
11:36:31 DEBUG: Parsing variable: time
11:36:31 DEBUG: Parsing variable: latitude
11:36:31 DEBUG: Parsing variable: longitude
11:36:31 DEBUG: Parsing variable: x_sea_water_velocity
11:36:31 DEBUG: Parsing varia

In [22]:
o.seed_elements(-123.6, 49, radius=100, time=daterange[0], number=100)

In [23]:
o.set_config('general:coastline_action', 'previous')
o.set_config('drift:scheme', 'runge-kutta4')
o.set_config('general:use_basemap_landmask', False)

In [24]:
o.run(time_step=90, time_step_output=3600, duration=timedelta(days=1))

11:36:34 DEBUG: 
------------------------------------------------------
Software and hardware:
  OpenDrift version 1.0.6
  251.8831558227539 GB memory
  32 processors (x86_64)
  Basemap version 1.1.0
  NumPy version 1.16.2
  SciPy version 1.2.1
  Matplotlib version 3.0.3
  NetCDF4 version 1.4.2
  Python version 3.7.3 (default, Mar 27 2019, 22:11:17) [GCC 7.3.0]
------------------------------------------------------

11:36:34 INFO: Config validation OK
11:36:34 DEBUG: No output file is specified, neglecting export_buffer_length
11:36:34 INFO: Fallback values will be used for the following variables which have no readers: 
11:36:34 INFO: 	x_wind: 0.000000
11:36:34 INFO: 	y_wind: 0.000000
11:36:34 INFO: Using existing reader for land_binary_mask
11:36:34 DEBUG: ----------------------------------------
11:36:34 DEBUG: Variable group ['land_binary_mask']
11:36:34 DEBUG: ----------------------------------------
11:36:34 DEBUG: Calling reader /data/bmoorema/results/opendrift/forcing/meshmask_

ValueError: Simulation stopped within first timestep. "Missing variables: ['land_binary_mask']", 'The simulation stopped before requested end time was reached.'


In [27]:
o

--------------------
Reader performance:
--------------------
/data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc
 0:00:00.0  total
 0:00:00.0  preparing
 0:00:00.0  reading
 0:00:00.0  interpolation
 0:00:00.0  interpolation_time
 0:00:00.0  masking
--------------------
Performance:
      0 total time
    4.1 configuration
    0.0 preparing main loop
      0.0 moving elements to ocean
      0.0 readers
        0.0 /data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc
        0.0 postprocessing
      0 main loop
--------------------
Model:	OceanDrift     (OpenDrift version 1.0.6)
	0 active PassiveTracer particles  (100 deactivated, 0 scheduled)
Projection: +proj=latlong
-------------------
Environment variables:
  -----
  land_binary_mask
  x_sea_water_velocity
  y_sea_water_velocity
     1) /data/bmoorema/results/opendrift/forcing/SalishSea_1h_20171110_20171110_opendrift.nc
  -----
Readers not added for the following vari

In [15]:
anim = o.animation()

In [16]:
HTML(anim.to_html5_video())

14:18:40 INFO: Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:18:40 INFO: figure size (inches) has been adjusted from 9.966152505214811 x 10.0 to 9.944444444444445 x 10.0
14:18:40 INFO: MovieWriter.run: running command: ['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '716x720', '-pix_fmt', 'rgba', '-r', '20.0', '-loglevel', 'quiet', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', '/tmp/tmpp6djj609/temp.m4v']
